In [2]:
#!pip install mediapipe-silicon opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 8.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 9.2 MB/s eta 0:00:0000:0100:01
  Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)


## Import

In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[1975]: Class CaptureDelegate is implemented in both /Users/bring/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x163476480) and /Users/bring/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1168a4860). One of the two will be used. Which one is undefined.
objc[1975]: Class CVWindow is implemented in both /Users/bring/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1634764d0) and /Users/bring/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11086ca68). One of the two will be used. Which one is undefined.
objc[1975]: Class CVView is implemented in both /Users/bring/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x1634764f8) and /Users/bring/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11086ca90). One of the two will be used. Which one is undefined.
objc[1975]: Class CVSlider is implemented in both /Users/bring/miniforge3/

## 1. Make Detections

### Capture real-time from the webcam

In [8]:
#Change the video capture

def make_1080p():
    cap.set(3, 1920)
    cap.set(4, 1080)

def make_720p():
    cap.set(3, 1280)
    cap.set(4, 720)

def make_480p():
    cap.set(3, 640)
    cap.set(4, 480)

def change_res(width, height):
    cap.set(3, width)
    cap.set(4, height)

## 2. Determine Joint

In [30]:
# VIDEO FEED
cap = cv2.VideoCapture(0)


make_720p()


## Setup mediapipe instance , min detection = 50% , min tracking = 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #Detect stuff and render
        #Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make detection
        results = pose.process(image)
        
        image.flags.writeable = True
                
        #Recolor image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks      
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=4, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=4, circle_radius=2)
                                 )
        
        

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cap.release()

In [5]:
for lnd in mp_pose.PoseLandmark:
    print(lnd)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


In [6]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7534187
y: 0.86692345
z: -0.3127252
visibility: 0.99754316

In [7]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.8586798
y: 1.3549049
z: -0.3893377
visibility: 0.10838135

In [8]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.8361805
y: 1.7303648
z: -0.75784105
visibility: 0.029700732

## 3. Calculate the Angles.

In [20]:
def calculate_angle(a,b,c):
    a = np.array(a) #First
    b = np.array(b) #Mid
    c = np.array(c) #End
    
    radian = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radian * 180/np.pi)
    
    if angle > 180:
        angle = 360 - angle
        
    return angle

In [21]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

[0.7534186840057373, 0.866923451423645]

In [15]:
calculate_angle(shoulder, elbow, wrist)

163.35067760928752

In [18]:
calculate_angle(hip, shoulder, elbow)

16.321276789536654

In [33]:
# VIDEO FEED
cap = cv2.VideoCapture(0)

make_720p()


## Setup mediapipe instance , min detection = 50% , min tracking = 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #Detect stuff and render
        #Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make detection
        results = pose.process(image)
        
        image.flags.writeable = True
                
        #Recolor image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks      
        try:
            #get coordinates
            landmarks = results.pose_landmarks.landmark
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            shoulder_angle = calculate_angle(hip, shoulder, elbow)
            #visualization
            angle = shoulder_angle
            cv2.putText(image, str(round(angle,3)), tuple(np.multiply(shoulder, [1280,720]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2, cv2.LINE_AA
                       )
        except:
            pass
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=4, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=4, circle_radius=2)
                                 )
        
        

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cap.release()

## 4. CURL Counter

In [35]:
# VIDEO FEED
cap = cv2.VideoCapture(0)

make_720p()

#Curl counter variables
counter = 0
stage = None




## Setup mediapipe instance , min detection = 50% , min tracking = 50%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        #Detect stuff and render
        #Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make detection
        results = pose.process(image)
        
        image.flags.writeable = True
                
        #Recolor image to BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks      
        try:
            #get coordinates
            landmarks = results.pose_landmarks.landmark
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            #shoulder_angle = calculate_angle(hip, shoulder, elbow)
            #visualization
            angle = elbow_angle
            cv2.putText(image, str(round(angle,3)), tuple(np.multiply(elbow, [1280,720]).astype(int)),
                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255),2, cv2.LINE_AA
                       )
            # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage == 'down':
                stage = "up"
                counter += 1
                print(counter)
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (255,73), (245, 117, 16), -1)
        #rep counter
        cv2.putText(image, 'REPS', (15,12),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        #Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=4, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=4, circle_radius=2)
                                 )
        
        

        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cap.release()

1
2
3
4
5
6
7
8
9
